In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
from pandas.io.json import json_normalize
%matplotlib inline

In [2]:
conn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
db = conn.get_database('maoyan')  # 连接到maoyan数据库

maoyan = db.get_collection('maoyan') # 连接到集合maoyan
mon_data = maoyan.find()  # 查询这个集合下的所有记录

In [3]:
data = json_normalize([comment for comment in mon_data])

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103716 entries, 0 to 103715
Data columns (total 11 columns):
_id           103716 non-null object
content       103716 non-null object
gender        103716 non-null int64
id            103716 non-null int64
nick          103716 non-null object
replyCount    103716 non-null int64
score         103716 non-null int64
time          103716 non-null int64
upCount       103716 non-null int64
userId        103716 non-null int64
userLevel     103716 non-null int64
dtypes: int64(8), object(3)
memory usage: 8.7+ MB


In [5]:
data.head()

,_id,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel
0,5c614a180d5a9ef918d0a0f5,很好震撼的很啊,1,1056560079,江生,0,9,1549879800000,0,201695017,3
1,5c614a180d5a9ef918d0a0f7,国产的科幻巅峰，丝毫不逊色好莱坞科幻大片，影响力就像星球大战一样，是国产科幻的开山之做，希望...,0,1056560077,Decadence,0,10,1549879800000,0,1020875871,4
2,5c614a180d5a9ef918d0a0f9,超级好看啊 户口帅的一批 我自己一个人看的 看得热血沸腾啊 爷爷死的时候真的哭死了,2,1056557517,暴走萝莉912,0,10,1549879800000,0,304299563,3
3,5c614a180d5a9ef918d0a0fb,保护生态环境，从我做,1,1056557516,*^_^*红尘逍遥,0,10,1549879800000,0,1548122992,2
4,5c614a180d5a9ef918d0a0fd,超级好看，感动，特效好哒,2,1056560066,罗大大,0,9,1549879800000,0,897364142,2


In [6]:
# 数据清洗
data = data.drop(columns='_id')
data = data.drop_duplicates(subset='userId')
data['time'] = pd.to_datetime(data['time']/1000, unit='s')
data = data[data['time']>=pd.to_datetime('2019-02-05 00:00:00')]
data.set_index(data["time"], inplace=True)
data.head()

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel
time,,,,,,,,,,
2019-02-11 10:10:00,很好震撼的很啊,1,1056560079,江生,0,9,2019-02-11 10:10:00,0,201695017,3
2019-02-11 10:10:00,国产的科幻巅峰，丝毫不逊色好莱坞科幻大片，影响力就像星球大战一样，是国产科幻的开山之做，希望...,0,1056560077,Decadence,0,10,2019-02-11 10:10:00,0,1020875871,4
2019-02-11 10:10:00,超级好看啊 户口帅的一批 我自己一个人看的 看得热血沸腾啊 爷爷死的时候真的哭死了,2,1056557517,暴走萝莉912,0,10,2019-02-11 10:10:00,0,304299563,3
2019-02-11 10:10:00,保护生态环境，从我做,1,1056557516,*^_^*红尘逍遥,0,10,2019-02-11 10:10:00,0,1548122992,2
2019-02-11 10:10:00,超级好看，感动，特效好哒,2,1056560066,罗大大,0,9,2019-02-11 10:10:00,0,897364142,2


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 102580 entries, 2019-02-11 10:10:00 to 2019-02-11 01:40:00
Data columns (total 10 columns):
content       102580 non-null object
gender        102580 non-null int64
id            102580 non-null int64
nick          102580 non-null object
replyCount    102580 non-null int64
score         102580 non-null int64
time          102580 non-null datetime64[ns]
upCount       102580 non-null int64
userId        102580 non-null int64
userLevel     102580 non-null int64
dtypes: datetime64[ns](1), int64(7), object(2)
memory usage: 8.6+ MB


#### 一、数据详情
- 共有102580条数据；
- 包含字段：评论内容、性别、评论ID、评论者昵称、回复数量、评分、时间、点赞数量、评论者ID、评论者等级

#### 二、问题
1. 总体评价如何？
2. 总体评价的时间走向如何？
3. 高分的评价理由是什么？
4. 低分的评价理由是什么？
5. 低分的人群有哪些特征？（性别、等级）
6. 低分跟哪位演员有关？

##### 1. 总体评价如何？

In [8]:
data['score'].mean()

9.241450575160851

In [9]:
from pyecharts import Bar
from pyecharts import Line
from pyecharts import Overlap

score_total = data['score'].value_counts().sort_index()
bar = Bar("《流浪地球》各评分数量", width=700)
line = Line("", width=700)
bar.add("", score_total.index, score_total.values, is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
line.add("", score_total.index, score_total.values+1000, is_smooth=True)

overlap = Overlap(width=700)
overlap.add(bar)
overlap.add(line)

overlap

In [10]:
# 低分百分比
score_total[:5].sum()/score_total.sum()*100

3.6342366933125363

In [11]:
# 高分百分比
score_total[7:].sum()/score_total.sum()*100

93.54650029245467

In [12]:
# 满分百分比
score_total[10:].sum()/score_total.sum()*100

71.40280756482746

##### 2. 总体评价的时间走向如何？

In [40]:
score_by_time = data['score'].resample('H').mean()
line = Line("《流浪地球》平均评分时间走向", width=700)
line.add("", score_by_time.index.date, score_by_time.values, is_smooth=True,
         legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18,
         xaxis_rotate=20, yaxis_min=8)
line

In [48]:
score_by_time.nsmallest(6)

time
2019-02-07 21:00:00    8.357143
2019-02-10 22:00:00    8.375000
2019-02-08 21:00:00    8.461538
2019-02-05 01:00:00    8.594203
2019-02-10 21:00:00    8.636364
2019-02-09 22:00:00    8.739130
Name: score, dtype: float64

##### 3. 高分的评价理由是什么？

In [103]:
import jieba
from collections import Counter
from pyecharts import WordCloud

jieba.add_word('屈楚萧')
jieba.add_word('刘启')
jieba.add_word('吴京')
jieba.add_word('刘培强')
jieba.add_word('李光洁')
jieba.add_word('王磊')
jieba.add_word('吴孟达')
jieba.add_word('达叔')
jieba.add_word('韩子昂')
jieba.add_word('赵今麦')
jieba.add_word('韩朵朵')

swords = [x.strip() for x in open ('stopwords.txt')]

In [132]:
def plot_word_cloud(data, swords):
    text = ''.join(data['content'])
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud

In [145]:
# 高分的评价
plot_word_cloud(data=data[data['score']>6], swords=swords)

In [146]:
data[data['score']>6].nlargest(10, 'upCount')

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel
time,,,,,,,,,,
2019-02-05 00:02:00,为什么没有7星，8星？完美已经不足以诠释这部电影的出色！感动，全程捏着拳头看完。叙事不拖沓，...,1,1053375466,奇异果Grissom,271,10,2019-02-05 00:02:00,5627,681463650,3
2019-02-05 11:28:00,在学校看了《流浪地球》的点映！除了电影情节、在现场看到了大刘老师！我更难忘的是郭帆导演在现场...,2,1052407417,Hope And Legacy,108,10,2019-02-05 11:28:00,3838,1812991790,4
2019-02-07 09:16:00,10分！太棒了！三次掉泪，父爱如山啊。40年前《星球大战》横空出世，好莱坞工业绿魔就此诞生；...,1,1054132930,CHAOSAY,29,10,2019-02-07 09:16:00,280,9693997,4
2019-02-05 14:56:00,超过预期。《流浪地球》的确开启了中国科幻片和灾难片的新纪元，无论是在哪一坐标中进行比对。虽然...,1,1053946211,徐若风,4,9,2019-02-05 14:56:00,224,116677026,4
2019-02-06 17:48:00,这场浩浩汤汤的太阳系“春运”燃情又暖心。父子情，战友情，人类与人工智能的情感与理智的抉择，电...,1,1054564832,电影大指南,12,10,2019-02-06 17:48:00,207,272340441,4
2019-02-05 11:01:00,中国终于有自己拿的出手的科幻片了!\n特效OK，剧情OK，没有大牌明星但全部演技都算得上OK...,0,1053741717,shadownihil,20,10,2019-02-05 11:01:00,201,8208177,4
2019-02-09 02:04:00,从故事层面，本片以刘慈欣科幻小说作为基底，剧本改编非常的扎实，不仅极具想象力，而且也涉及到很...,1,1055541594,娱评天下,9,10,2019-02-09 02:04:00,184,172967358,4
2019-02-05 13:03:00,剧作短板，特效完美，演员优秀，逻辑还是完全经不起推销。地球离太阳最后的距离还不得冻成一个冰球...,2,1053840774,刘小黛,194,7,2019-02-05 13:03:00,174,12695322,3
2019-02-05 14:44:00,前部分虽有不足，但特效是超出预期，后劲较足，到了中后部分有两个地方感动到我，刘培强实现预言，...,2,1053940077,JuliaFanl,6,8,2019-02-05 14:44:00,172,227360759,5


In [147]:
for i in data[data['score']>6].nlargest(10, 'upCount')['content']:
    print(i+'\n')

为什么没有7星，8星？完美已经不足以诠释这部电影的出色！感动，全程捏着拳头看完。叙事不拖沓，没有美式灾难片的个人英雄主义；宏大，还是宏大，全片充满各种想象；特效，丝毫不输好莱坞进口大片，甚至有些地方有过之而无不及。强烈建议，制作团队赶紧买了三体的版权吧，三年、五年甚至十年，我都等……

在学校看了《流浪地球》的点映！除了电影情节、在现场看到了大刘老师！我更难忘的是郭帆导演在现场讲述的拍摄历程。真的真的太难了、也许有的人会一听国产科幻失去兴趣（真的，我身边哪怕看了很多书的同学、也有的会这样）这也不能怪任何人，毕竟大家看电影也都是想看到好看的、但是对于对科幻有兴趣的朋友，真的来看一看吧。我相信同为科幻迷并为科幻奋斗了这么多年的郭帆老师、绝对值得一张电影票
    在看后
太让人惊喜了，紧凑的节奏、多次的剧情反转、
以及我们自己的价值观（如果是美国电影、大概最后会采用让一部分精英去往新家园，而我们中国的农耕文明注定了我们对土地对地球的情感无法割舍，在流浪地球背后正是这层含义吧

10分！太棒了！三次掉泪，父爱如山啊。40年前《星球大战》横空出世，好莱坞工业绿魔就此诞生；而在40年后，《流浪地球》的上映，标志着中国科幻片就此诞生了。稳稳的拿下中国科幻片里程碑的称号了。再渺小的生命， 只要心有定力， 终会达到目的。要有牺牲小我，完成大我的勇气，小人物也是能拯救世界的。

超过预期。《流浪地球》的确开启了中国科幻片和灾难片的新纪元，无论是在哪一坐标中进行比对。虽然我们并非是一直都没有所谓的科幻电影，但其中的大多都不是工业化、硬科幻的路子。
《流浪地球》则在概念和执行上都达到了这一标准，特效技术、工整剧作、演员所合力营造出的氛围都很对。它代表的，是华语科幻电影的一个“关键转折点”。
从原著角度而言，这是一个带着孤独感的故事。电影采用了小说中的整体设定，但故事与人物有了新的创造，且塑造了很好的主题，即对乡土情结的回归和牺牲精神。

这场浩浩汤汤的太阳系“春运”燃情又暖心。父子情，战友情，人类与人工智能的情感与理智的抉择，电影赋予了刘慈欣原著更多没有的元素。张弛有度的节奏，两个小时的时间，心情随着剧情起伏。更别说激动人心，艳惊四座的特效画面了。看过《流浪地球》，我们终于可以理直气壮的说:国产电影终于有了真正的硬核科幻片！

中国终于有自己拿的出手的科幻片了!
特效OK，剧情OK，没有

##### 4. 低分的评价理由是什么？

In [148]:
# 低分的评价
plot_word_cloud(data=data[data['score']<5], swords=swords)

In [149]:
data[data['score']<5].nlargest(10, 'upCount')

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel
time,,,,,,,,,,
2019-02-06 23:47:00,也许进入地下没有了阳光，人们的内心就趋向黑暗！小太妹一样的妹妹人设，不知所谓的混血？没有缘由...,1,1054591874,逗逼老爸,67,2,2019-02-06 23:47:00,99,163219720,4
2019-02-07 23:00:00,太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分...,1,1055063063,杨某X,42,2,2019-02-07 23:00:00,95,85500030,1
2019-02-08 04:14:00,电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的...,1,1055119853,一抹晨曦784,21,3,2019-02-08 04:14:00,75,156486326,2
2019-02-08 01:42:00,情感细节0分，演员演技0分，特别是17岁的那两个孩子，真是醉了！从前烂到尾，bug多如牛毛。...,1,1055075620,秭归小武,25,1,2019-02-08 01:42:00,75,70329844,1
2019-02-08 19:51:00,没逻辑没智商没演技超尴尬！最后不作不死的小女生用一篇激情澎湃的即兴小学生作文感动了全球救援队...,1,1055518330,威仔,22,1,2019-02-08 19:51:00,74,419136037,3
2019-02-08 02:18:00,我想问问你们摸着良心说是真的觉得好看吗？反正有不少人跟我一起提前离场就是了。,0,1055083535,弧鸽,38,1,2019-02-08 02:18:00,74,1008203632,2
2019-02-06 06:16:00,理性分析，我是科幻迷，我对这部电影的科幻背景没有一点意见，特效当然是进步了，但用力过度，剧情...,1,1054184286,Refine？,97,4,2019-02-06 06:16:00,72,1053444729,2
2019-02-07 12:18:00,看过好后悔，好奇怪怎么会有这么多好评？真的不太好看，主演真心是败笔！几个年轻孩子根本撑不起这...,2,1054680274,淡然1547,48,3,2019-02-07 12:18:00,70,45444833,2
2019-02-10 10:40:00,这么烂的电影居然这么高评分，可见水军力量可以，还有就是随波逐流的那些不懂电影的人给个特效的赞...,1,1055061178,栄栄栄轩,25,1,2019-02-10 10:40:00,69,48197366,2


In [150]:
for i in data[data['score']<5].nlargest(10, 'upCount')['content']:
    print(i+'\n')

也许进入地下没有了阳光，人们的内心就趋向黑暗！小太妹一样的妹妹人设，不知所谓的混血？没有缘由的痞帅（真心不帅），地震只顾自己逃亡不让救人的姥爷，吐满自己的头盔又抢别人头盔的救援队？为了亲情强行不进入休眠的爸爸，为了不再死人打坏火石的女队员……不学无术的少年比世界所有科学家都厉害，看来地球就剩下牛顿的弟弟牛逼啦！整部片子充满了一种戾气！为啥中国拯救世界就非得用这样的人物支撑人设？为了后面的反转？虽然特效还行，真心不好看！希望不要影响大家的观影心情！

太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分样子，在国产片里面算不错的，剧情烂，东拉西扯的，地球你造发动机推动脱离太阳引力我也忍了，点燃木星就太扯了，点燃木星瞬间推动地球逃脱木星引力，那么大的冲击力地球生物不死绝鬼都不信！还有作为灾难科幻片，震撼人心的灾难场景基本没有，地球停海啸爆发，我一直在期待灾难爆发时刻人类逃亡的场景，但是从头到尾都没有！走到哪里都挂着国旗，又是打爱国情怀，很符合吴京蹭爱国热度的习惯，反正不会错，但是我就觉得很作！恶心！

电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的有气势，但是演员演技实在捉急，特别是两个主角，男主人设叛逆性格太过了，女主那段演讲是本片的最大败笔，看完那段演讲让我尴尬的有种跑出电影院的冲动，每个人物死掉都会莫名其妙的来段演讲煽情戏，编剧真的是尬王！总的来说，网上给予这片子得评分没有很客观，相比较国外科幻大片，流浪地球有进步，但是还差得远！电影就是电影跟爱不爱国无关，请不要道德绑架！

情感细节0分，演员演技0分，特别是17岁的那两个孩子，真是醉了！从前烂到尾，bug多如牛毛。无数个人在玩手机。中国科幻电影需要学习的地方还有太多太多。

以后看电影不能再去豆瓣上看评分啦，前期的水军真多！

没逻辑没智商没演技超尴尬！最后不作不死的小女生用一篇激情澎湃的即兴小学生作文感动了全球救援队！成功拯救了地球！这导演跟编剧是小学没毕业吧！

我想问问你们摸着良心说是真的觉得好看吗？反正有不少人跟我一起提前离场就是了。

理性分析，我是科幻迷，我对这部电影的科幻背景没有一点意见，特效当然是进步了，但用力过度，剧情从引爆木星那段就开始崩了，在此之前一切都称得上非常不错，在引爆木星剧情开始前，我甚至以为电影

##### 5. 低分的人群有哪些特征？（性别、等级）

In [172]:
# 总体的性别比例
gender_total = data['gender'].value_counts()
bar = Bar("《流浪地球》观众性别", width=700)
bar.add("", ['未知', '男', '女'], gender_total.values, is_stack=True, is_label_show=True,
       bar_category_gap='60%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [175]:
gender_total/gender_total.sum()*100

0    43.298889
1    36.886333
2    19.814779
Name: gender, dtype: float64

In [185]:
# 低分的性别比例
gender_low = data.loc[data['score']<5, 'gender'].value_counts()
bar = Bar("《流浪地球》低分评论观众性别", width=700)
bar.add("", ['未知'zh, '男', '女'], gender_low.values, is_stack=True, is_label_show=True,
       bar_category_gap='60%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [179]:
gender_low/gender_low.sum()*100

0    42.650215
1    37.821888
2    19.527897
Name: gender, dtype: float64

In [187]:
# 总体的等级比例
level_total = data['userLevel'].value_counts().sort_index()
bar = Bar("《流浪地球》观众等级", width=700)
bar.add("", level_total.index, level_total.values, is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [188]:
# 低分评论的观众等级比例
level_low = data.loc[data['score']<5, 'userLevel'].value_counts().sort_index()
bar = Bar("《流浪地球》低分评论的观众等级", width=700)
bar.add("", level_low.index, level_low.values, is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

##### 6. 高低分跟哪位演员有关？

In [198]:
mapping = {'liucixin':'刘慈欣|大刘', 'guofan':'郭帆', 'quchuxiao':'屈楚萧|刘启|户口', 'wujing':'吴京|刘培强', 
           'liguangjie':'李光洁|王磊', 'wumengda':'吴孟达|达叔|韩子昂', 'zhaojinmai':'赵今麦|韩朵朵'}
for key, value in mapping.items():
    data[key] = data['content'].str.contains(value)

In [204]:
# 总体提及次数
staff_count = pd.Series({key: data.loc[data[key], 'score'].count() for key in mapping.keys()}).sort_values()
staff_count

liguangjie     171
guofan         272
zhaojinmai     377
wumengda       477
quchuxiao      833
liucixin      3122
wujing        5351
dtype: int64

In [205]:
bar = Bar("《流浪地球》演职员总体提及次数", width=700)
bar.add("", ['李光洁','郭帆','赵今麦','吴孟达','屈楚萧','刘慈欣','吴京'], staff_count.values, is_stack=True, is_label_show=True,
       bar_category_gap='60%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [215]:
average_score = pd.Series({key: data.loc[data[key], 'score'].mean() for key in mapping.keys()}).sort_values()
average_score

zhaojinmai    8.636605
wumengda      8.779874
wujing        9.003738
quchuxiao     9.018007
liguangjie    9.134503
liucixin      9.480782
guofan        9.658088
dtype: float64

In [219]:
bar = Bar("《流浪地球》演职员平均分", width=700)
bar.add("", ['赵今麦','吴孟达','吴京','屈楚萧','李光洁','刘慈欣','郭帆'], np.round(average_score.values,2), is_stack=True, is_label_show=True,
       bar_category_gap='60%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [220]:
staff_count_low = pd.Series({key: data.loc[data[key]&(data['score']<5), 'score'].count() for key in mapping.keys()}).sort_values()
staff_count_low

guofan          3
liguangjie      4
zhaojinmai     14
wumengda       29
quchuxiao      30
liucixin       36
wujing        305
dtype: int64

In [231]:
staff_count_pct = np.round(staff_count_low/staff_count*100, 2).sort_values()
staff_count_pct

guofan        1.10
liucixin      1.15
liguangjie    2.34
quchuxiao     3.60
zhaojinmai    3.71
wujing        5.70
wumengda      6.08
dtype: float64

In [233]:
bar = Bar("《流浪地球》演职员低分评论提及百分比", width=700)
bar.add("", ['郭帆','刘慈欣','李光洁','屈楚萧','赵今麦','吴京','吴孟达'], staff_count_pct.values, is_stack=True, is_label_show=True,
       bar_category_gap='60%', label_color = ['#130f40'],
       legend_text_size=18,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [229]:
data[data['wumengda']&(data['score']<5)].nlargest(5, 'upCount')

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,,
2019-02-05 14:11:00,可惜了60+大洋，达叔的演技是唯一的亮点，小鲜肉们稚嫩且乏味的演技（约等于0.1）好无聊，和...,1,1053902705,熊晓军［德恒律师］,13,1,2019-02-05 14:11:00,42,1154499307,5,False,False,False,False,False,True,False
2019-02-11 02:14:00,真的，这一分我都是给的吴孟达和雷佳音的，剩下的真的不能多给了，电影，最基本的是故事情节，一味...,2,1056402038,林艾G,52,1,2019-02-11 02:14:00,40,63455639,3,False,False,False,False,False,True,False
2019-02-08 04:46:00,主人公太扯了，三观不正，净瞎拍，拿着吴京吴孟达做幌子，真浪费我一百大洋。这片子也就骗骗小孩能...,1,1055139154,博爱永恒,11,3,2019-02-08 04:46:00,39,53086905,2,False,False,False,True,False,True,False
2019-02-09 04:33:00,也别给半星了，剧情太烂，情节不好，故事不合理，尤其最后爆炸，还能爆炸5000公里，飞船都让你...,1,1055580599,闹闹先生丶丶,18,2,2019-02-09 04:33:00,31,173755020,3,False,False,False,False,False,True,False
2019-02-07 10:19:00,台词设定太尴尬，强行煽情，那个外国人设定最恶心，强行设定的人物，吴京，吴孟达只是客串，还有那...,0,1054828448,A_啊杰,7,1,2019-02-07 10:19:00,28,1033874459,4,False,False,False,True,False,True,False


In [236]:
for i in data[data['wumengda']&(data['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

可惜了60+大洋，达叔的演技是唯一的亮点，小鲜肉们稚嫩且乏味的演技（约等于0.1）好无聊，和看昨晚的春晚感觉差不多，好像喊得很大声，哭得很撕心，但就是没内容，演得太空洞。催泪不够真诚，搞笑不够认真，细节太过毛糙，整个就是一部被制作，为制作而制作的视听片，片尾曲让人无力吐槽。一个人的初一良辰就这么被拱了。

真的，这一分我都是给的吴孟达和雷佳音的，剩下的真的不能多给了，电影，最基本的是故事情节，一味强调特技多么好多么好，去看科技展不就好了？真的是浪费了80块的电影票，毫无情节可言，一味地个人英雄主义，强行煽情，把观众得智商按在地上摩擦，这样的烂片是买了水军吧……垃圾！

主人公太扯了，三观不正，净瞎拍，拿着吴京吴孟达做幌子，真浪费我一百大洋。这片子也就骗骗小孩能行，真的不敢恭维。

也别给半星了，剧情太烂，情节不好，故事不合理，尤其最后爆炸，还能爆炸5000公里，飞船都让你开这么快了，还用的着点燃么？直接开进木星不就好了，飞船后面喷的不是火啊？送达叔一张票，卖战狼一份情，这电影真不行

台词设定太尴尬，强行煽情，那个外国人设定最恶心，强行设定的人物，吴京，吴孟达只是客串，还有那人物设定，太牵强，不知道9.3分怎么来的



In [234]:
data[data['wujing']&(data['score']<5)].nlargest(5, 'upCount')

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,,
2019-02-07 23:00:00,太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分...,1,1055063063,杨某X,42,2,2019-02-07 23:00:00,95,85500030,1,False,False,False,True,False,False,False
2019-02-06 17:21:00,逻辑非常不严密，既然定位是科幻片，起码的逻辑得有吧？\n1.一个0.7级宇宙文明等级的星球，...,1,1054564334,团拼丶小王子,108,1,2019-02-06 17:21:00,58,8675410,3,False,False,False,True,False,False,False
2019-02-07 03:59:00,就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什...,2,1054646769,CpJ26143934,35,4,2019-02-07 03:59:00,53,91570358,2,False,False,True,True,False,False,True
2019-02-07 03:22:00,开头以为是国产科幻大片，气象不凡，道具方面下了不少功夫，某些场景直追好莱坞。整体上离好莱坞还...,1,1054637652,meituanchzhh,33,1,2019-02-07 03:22:00,49,34052306,3,False,False,False,True,False,False,False
2019-02-05 09:17:00,什么玩意儿，发动机和石头砸下来的声音从头震到尾，演技拙劣的儿女情长的片段令人作呕，吴京一副大...,0,1053645922,westbrookFMVP,36,1,2019-02-05 09:17:00,48,779520607,2,False,False,False,True,False,False,False


In [235]:
for i in data[data['wujing']&(data['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分样子，在国产片里面算不错的，剧情烂，东拉西扯的，地球你造发动机推动脱离太阳引力我也忍了，点燃木星就太扯了，点燃木星瞬间推动地球逃脱木星引力，那么大的冲击力地球生物不死绝鬼都不信！还有作为灾难科幻片，震撼人心的灾难场景基本没有，地球停海啸爆发，我一直在期待灾难爆发时刻人类逃亡的场景，但是从头到尾都没有！走到哪里都挂着国旗，又是打爱国情怀，很符合吴京蹭爱国热度的习惯，反正不会错，但是我就觉得很作！恶心！

逻辑非常不严密，既然定位是科幻片，起码的逻辑得有吧？
1.一个0.7级宇宙文明等级的星球，自己这块地的资源分配都没整明白，是哪来的能量去推着地球改变行星轨迹？
2.点燃木星这脑洞是有多大？几十亿年有多少陨石彗星冲击都没点燃，你突发奇想放个火就能点燃了？全是氢气，是怎么点燃的？就这种气体混合比，怎么点燃的？麻烦把化学反应方程式告诉我一下。还不如说丢个炮仗上去炸一下，起码炮仗还能在无氧的环境下燃烧爆炸。
3.就吴京这种个人英雄主义的演员拿来演这种格局庞大的科幻电影合适吗？单说人物设定，就这种人的军衔是怎么给的？是怎么选为航天员的？有没有做过心理测试？有没有做过性格评估？人类的命运最后是堵出来的？

就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什么都是别人的错，对着姥爷一口一个老东西，姥爷临死也没改口；韩朵朵就是一**少女，紧要关头还声情并茂朗读小作文；刘培强是我要救儿子……以为儿子死了：好，我放弃了，我听moss的回去休眠……哦！儿子没死，太好了，我要想办法，一定要救地球……🙄️🙄️🙄️

开头以为是国产科幻大片，气象不凡，道具方面下了不少功夫，某些场景直追好莱坞。整体上离好莱坞还是差半个世纪。1⃣️演员。格式化表演，看不出“人”性内容，几处人物牺牲逻辑性和共鸣性不强，尤其吴京最后牺牲前一段烈士般台词，沿习战狼做派，略显矫情。2⃣️布景。感觉一直都是漫天飞雪、错杂纷乱，缺乏变化。整体风格压抑，没有情景沉入感、带入感。

什么玩意儿，发动机和石头砸下来的声音从头震到尾，演技拙劣的儿女情长的片段令人作呕，吴京一副大义凛然的样子太过难受，什么中国第一部科幻，情节稀烂，特效主要靠音效，加上刻意塑造的中国式英雄，一分没得说



In [237]:
data[data['zhaojinmai']&(data['score']<5)].nlargest(5, 'upCount')

,content,gender,id,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,,
2019-02-07 03:59:00,就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什...,2,1054646769,CpJ26143934,35,4,2019-02-07 03:59:00,53,91570358,2,False,False,True,True,False,False,True
2019-02-06 15:46:00,刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，...,2,1054533126,pSU598729902,23,1,2019-02-06 15:46:00,35,1054238164,3,True,False,False,False,False,False,True
2019-02-05 05:53:00,能更尴尬么？韩朵朵尴尬，刘启尴尬，演技尴尬，剧情尴尬，MOSS 反转尴尬，除了三体 再也不看...,0,1053496256,刀客谭,21,1,2019-02-05 05:53:00,33,1010202977,3,True,False,True,False,False,False,True
2019-02-10 12:56:00,不感动啊…也没有满场子哭泣泣的！不过制作看得出来很精致，但独白很一般！韩朵朵喊救援时候太拖拉...,2,1056241443,崔家内小谁,0,4,2019-02-10 12:56:00,12,199632141,4,False,False,False,False,False,False,True
2019-02-10 16:33:00,男权主义太重了，我真的十分讨厌吧危机关头，把女生演绎的像个傻子，赖赖唧唧的，拖后腿，最后几分...,2,1055645215,@,3,3,2019-02-10 16:33:00,2,1045118082,2,False,False,False,False,False,False,True


In [238]:
for i in data[data['zhaojinmai']&(data['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什么都是别人的错，对着姥爷一口一个老东西，姥爷临死也没改口；韩朵朵就是一**少女，紧要关头还声情并茂朗读小作文；刘培强是我要救儿子……以为儿子死了：好，我放弃了，我听moss的回去休眠……哦！儿子没死，太好了，我要想办法，一定要救地球……🙄️🙄️🙄️

刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，镜头语言幼稚，几乎全程在快速地晃荡，人物情感没有张力，演员功力都弱弱的，都末日了，还不咸不淡的语气。。男主对他爹的大恨是从哪儿来的？？不就是母亲放弃治疗吗？那个韩朵朵出来是干什么的？是让大家保护的“希望”化身？？强行煽情的台词让我汗毛竖起。刻意卖的末世日情怀，我不想接。狂热的拥护。

原来酒带上太空舱是为了炸掉智能机器人莫斯的。。被酒燃烧前莫斯的话是经典——想让人类保持理智是一种奢望。笑岔我。

看到东方明珠塔被雪冻住了我就忍不住笑，不知道老美看到纽约被炸那么多次什么心情。

无知导致浪漫。



能更尴尬么？韩朵朵尴尬，刘启尴尬，演技尴尬，剧情尴尬，MOSS 反转尴尬，除了三体 再也不看刘慈欣的任何电影。失望。

不感动啊…也没有满场子哭泣泣的！不过制作看得出来很精致，但独白很一般！韩朵朵喊救援时候太拖拉了！我都着急了！

男权主义太重了，我真的十分讨厌吧危机关头，把女生演绎的像个傻子，赖赖唧唧的，拖后腿，最后几分钟通话光听她哭了，我叫韩朵朵，麻烦你想煽情的同时，台词有点说服力可以吗？4分全给特效，编、导、演都很差

